#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-1（理論編）

##はじめに
2020年の初頭現在で実用的に量子コンピュータで解ける問題は「変分アルゴリズム」と呼ばれるVQEやQAOAと言った量子コンピュータと古典コンピュータをハイブリッドで利用したものが主流になっています。ここでは実用性を重視して、これらの量子古典ハイブリッドアルゴリズムの使い方を見てみたいと思います。

###概要
量子古典ハイブリッドアルゴリズムのVQE(Variational Quantum Eigensolver)は、現在の量子コンピュータはエラーがまだ多いため、従来利用されると思われていた「位相推定」アルゴリズムの代替として開発され、2013年に当時ハーバード大学（現在はトロント大学）のアラン・アスプル・グジック教授のチームによって開発されました。

現在の量子コンピュータはエラーが多く、長い量子回路を組むとエラーが蓄積し正しい答えを得ることができません。想定されていたアルゴリズムは全て長い回路が基本となっていました。それに対して、短い量子回路をたくさん計算し、それを集計し、最適化をかけることで同じ機能を実現するVQEアルゴリズムが現在の主流となっています。

<img width="489" alt="スクリーンショット 2020-02-29 14.42.40.png" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/f4f0b63b-0bc9-6f26-cacb-58cfd44f02c7.png">

##固有値・固有ベクトル問題
VQEで解ける問題は、「固有値・固有ベクトル問題」と呼ばれるもので、ある行列に対する固有値と固有ベクトルを見つけることができれば様々な問題を解くことができます。ある与えられた行列$H$に対して、固有値を$E_0$、固有ベクトルを$\mid \psi \rangle$としたときに、

$$H \mid \psi \rangle = E_0 \mid \psi \rangle$$

を満たすような、$E_0$と$\psi$を見つけるのが目的です。$H$はハミルトニアン、$\mid \psi \rangle$は状態ベクトル（波動関数）と呼ばれ、状態ベクトルの固有ベクトルのことを固有状態と呼びます。

##実用的に解ける問題
VQEは汎用的なアルゴリズムでハミルトニアン$H$が与えられれば本来どのような固有値・固有ベクトルも探せるはずですが、実際には最適化計算で探索するパラーメータの範囲がとても広いため、与えられたハミルトニアン$H$と求める固有ベクトルである$\mid \psi \rangle$の間にある程度の関連性がないと実質的に解くことができません。

2020年初頭現在で解くことができると言われているのが、「量子化学計算」と「組合せ最適化計算」と呼ばれる２種類で、それぞれ与えられたハミルトニアン$H$と求めたい固有状態$\mid \psi \rangle$の間に関連性を見出すことができ、それらをある程度定式化することができています。

上記の固有状態を作り出すためには量子回路を組んで、その結果として固有状態$\mid \psi \rangle$を得ますが、この量子回路のことを特に、「ansatz（アンザッツ）」と呼びます。量子化学では、UCCansatzと呼ばれるUCC(Unitary Coupled Cluster)理論を利用しansatzを構成し、組合せ最適化では、QAOAansatzと呼ばれるQAOA(Quantum Approximate Optimization Algorithm)理論を利用してansatzを組み立てます。

実質的に行っていることは似通っていますが、特に後者はQAOAという名前で呼ばれることが通常です。ここでは、VQEとQAOAを中心に理論の下準備と実際の活用をblueqatSDK( https://github.com/blueqat/blueqat )を利用して解説していきます。

##変分アルゴリズム2-1（理論編）
手計算を中心に理論を確認します。主に変分アルゴリズムと呼ばれるVQEとQAOAはほぼ同じ手順で実行をすることが可能です。苦手な方はこの章は飛ばしても結構です。主に学ぶ内容は、

・ハミルトニアンについて  
・ansatzについて

の二つです。最後に簡単な例題を実行してみます。

##blueqatのインストール
今後例題にはblueqatを使います。

https://github.com/Blueqat/Blueqat

インストールは、

In [1]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


で、完了します。

##2-1-1 下準備１：量子ビットと状態ベクトル
量子コンピュータで利用される量子ビットは|0>状態と|1>状態を持っていてベクトルで表現できます。

$$
\mid 0 \rangle =
\begin{bmatrix}
1 \\
0
\end{bmatrix},
\mid 1 \rangle =
\begin{bmatrix}
0 \\
1
\end{bmatrix}
$$

また、任意の量子ビットは重ね合わせて量子状態を表現できます。


$$
\mid \psi \rangle = \alpha \mid 0 \rangle + \beta \mid 1 \rangle =
\alpha 
\begin{bmatrix}
1 \\
0
\end{bmatrix}
+ \beta
\begin{bmatrix}
0 \\
1
\end{bmatrix}
=
\begin{bmatrix}
\alpha \\
\beta
\end{bmatrix}
$$

$\mid \psi \rangle$は状態ベクトル(波動関数）と呼ばれ量子ビットの量子状態を表します。$\alpha$と$\beta$は複素数で「確率振幅」と呼ばれ、二乗するとその対応する計算基底の出現確率になり、$|\alpha|^2$や$|\beta|^2$は「測定」をすることで取得できます。また、

$$
|\alpha|^2+|\beta|^2 = 1
$$

を満たします。例えば、100回計算して、0が45回、1が55回でたら、$|\alpha|^2$の期待値は0.45、$|\beta|^2$の期待値は0.55と計算できます。実際の状態ベクトルは直接測定をすることはできないため、通常は何度も計算を実行して統計的なサンプルを取ります（シミュレータでは直接状態ベクトルを取る機能もありますが、サイズの大きいものは取れません）。

##2-1-2 下準備2：状態ベクトルの操作と量子ゲート
状態ベクトルを操作するには、量子ゲートを利用します。変分アルゴリズムでは基本のパウリゲート、任意回転ゲート、アダマールゲートを主に利用します。

パウリゲートはXYZの３種類があり、それぞれブロッホ球でのXYZ軸周りでの180度の回転を意味し、対応する行列は、

$$
X=
\begin{bmatrix}
0&1 \\
1&0
\end{bmatrix},
Y=
\begin{bmatrix}
0&-i \\
i&0
\end{bmatrix},
Z=
\begin{bmatrix}
1&0 \\
0&-1
\end{bmatrix}
$$

となります。また、XYZ軸周りに任意回転にしたのが、RX,RY,RZゲートで、

$$
Rx(\theta) = \left( \begin{array}{cc} \cos\left(\frac{\theta}{2}\right) &
-i\sin\left(\frac{\theta}{2}\right)\\
-i\sin\left(\frac{\theta}{2}\right) &
\cos\left(\frac{\theta}{2}\right) \end{array} \right),
Ry(\theta) = \left( \begin{array}{cc} \cos\left(\frac{\theta}{2}\right) &
-\sin\left(\frac{\theta}{2}\right)\\
\sin\left(\frac{\theta}{2}\right) &
\cos\left(\frac{\theta}{2}\right) \end{array} \right),
Rz(\theta) = \left( \begin{array}{cc} e^{-i\frac{\theta}{2}} & 0\\ 0 & e^{i\frac{\theta}{2}} \end{array} \right)
$$

となります。アダマールゲートは、

$$
H =
\frac{1}{\sqrt{2}} 
\begin{bmatrix}
1&1 \\
1&-1
\end{bmatrix}
$$

となります。主にX軸とZ軸の間で変換を行います。状態ベクトルにこれらの量子ゲートを作用させることで量子状態を変化させることができます。

$$
X
\begin{bmatrix}
1 \\
0
\end{bmatrix}
=
\begin{bmatrix}
0&1 \\
1&0
\end{bmatrix}
\begin{bmatrix}
1 \\
0
\end{bmatrix}
=
\begin{bmatrix}
0 \\
1
\end{bmatrix}
$$

##2-1-3 下準備３：複数量子ビットの状態ベクトルとテンソル積
上記の状態ベクトルは１量子ビットの時のものです。２量子ビット以上の状態ベクトルはテンソル積を使って表現します。

$$
\begin{bmatrix}
a \\
b
\end{bmatrix}
\otimes
\begin{bmatrix}
c \\
d
\end{bmatrix}
=
\begin{bmatrix}
a*
\begin{bmatrix}
c\\
d
\end{bmatrix}
 \\
b*
\begin{bmatrix}
c\\
d
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ac \\
ad \\
bc \\
bd
\end{bmatrix}
$$

３量子ビット以上の場合も同様です。

$$
\begin{bmatrix}
a \\
b
\end{bmatrix}
\otimes
\begin{bmatrix}
c \\
d
\end{bmatrix}
\otimes
\begin{bmatrix}
e \\
f
\end{bmatrix}
=
\begin{bmatrix}
ac *
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
ad*
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
bc*
\begin{bmatrix}
e \\
f
\end{bmatrix} \\
bd*
\begin{bmatrix}
e \\
f
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ace\\
acf\\
ade\\
adf\\
bce\\
bcf\\
bde\\
bdf
\end{bmatrix}
$$

量子ゲートもテンソル積をとれます。ハミルトニアンの計算の際に利用します。

$$
\begin{bmatrix}
a&b \\
c&d
\end{bmatrix}
\otimes
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix} \\
=
\begin{bmatrix}
a*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&b*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\\
c*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
&d*
\begin{bmatrix}
e&f \\
g&h
\end{bmatrix}
\end{bmatrix}
=
\begin{bmatrix}
ae&af&be&bf\\
ag&ah&bg&bh\\
ce&cf&de&df\\
cg&ch&dg&dh
\end{bmatrix}
$$


##2-1-4 量子変分原理
今回固有値を直接求めるのは厳しいので、間接的に固有値の期待値にできるだけ近づける最適化計算に落とし込みます。量子変分原理より、任意の状態ベクトルにおけるハミルトニアンの期待値は基底状態を上回るというのがあります。

$$
\langle \Psi (\theta) \mid H \mid \Psi (\theta) \rangle \geq E_0
$$

これは、与えられたハミルトニアン$H$に対して任意の状態ベクトルは常に最小の固有値の期待値を上回るということを示しています。ハミルトニアン$H$は与えられた式なので、私たちにできるのは状態ベクトル$\mid \Psi (\theta) \rangle$をできるだけ固有状態に近づけるということです。これにより、できるだけ固有状態に近い状態ベクトルを角度の最適化計算として探していくのが今回の目的です。

##2-1-5 変分アルゴリズムの計算手順
変分アルゴリズムの計算手順は、

１、パラメータ$\theta$を導入した量子回路（ansatz）を準備（量子計算）
２、量子ビットの測定値からハミルトニアン$H$の期待値$\langle \Psi (\theta) \mid H \mid \Psi (\theta) \rangle$を計算（古典計算）
３、古典最適化計算により、ハミルトニアン$H$の期待値を最小にするパラメータ$\theta$を探索（古典計算）

のように、量子計算と古典計算のハイブリッドで成り立っています。

<img width="489" alt="スクリーンショット 2020-02-29 14.42.40.png" src="https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/218694/f4f0b63b-0bc9-6f26-cacb-58cfd44f02c7.png">

引用：https://arxiv.org/pdf/1304.3061.pdf

注意点は、VQEは量子回路はハミルトニアン$H$がユニタリ行列の場合の期待値しか計算できません。ハミルトニアン$H$は通常エルミート行列で与えられるので、ユニタリ行列の和に分解する必要があります。


##2-1-6 ansatz（アンザッツ）
ansatz（アンザッツ）は、状態ベクトルを作るための、主に任意回転ゲートでパラメータ化された量子回路で、ハミルトニアン$H$の期待値を求めるための測定値を提供します。

##2-1-7 ハミルトニアン
ハミルトニアン$H$はパウリゲートXYZと単位行列Iから構成される行列です。今回の変分アルゴリズムは、このハミルトニアン$H$の固有状態である状態ベクトルから導き出される固有値の期待値を求めることを目的としています。

##2-1-8 ハミルトニアンの期待値
ある状態ベクトル$\mid \psi \rangle$とハミルトニアン$H$が与えられた時のハミルトニアン$H$の期待値は、

$$
\langle \psi (\theta) \mid H \mid \psi (\theta) \rangle
$$

の簡単な計算によってもとまります。


##2-1-9 例題：１量子ビットのansatzを使ってハミルトニアンZの期待値を求める
ansatzを使った任意の状態ベクトルにおけるハミルトニアンが$H = Z$の場合の期待値は、

$$
\langle \psi \mid H \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
1&0\\
0&-1
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= |\alpha|^2 - |\beta|^2
$$

となります。ここで、$|\alpha|^2$は0がでる確率、$|\beta|^2$は1が出る確率に対応します。

##2-1-10 例題２：１量子ビットのansatzを使ってハミルトニアンXの期待値を求める
ハミルトニアンが$H = X$の場合のハミルトニアン$H$の期待値は、

$$
\langle \psi \mid H \mid \psi \rangle = 
\begin{bmatrix}
\alpha^* & \beta^*
\end{bmatrix}
\begin{bmatrix}
0&1\\
1&0
\end{bmatrix}
\begin{bmatrix}
\alpha\\
\beta
\end{bmatrix}
= \alpha^* \beta + \alpha \beta^*
$$

となりますが、通常の測定ではこの値を求めることができませんので、ハミルトニアンの変形を通じて解を得ます。



##2-1-11 ハミルトニアンの変形
ハミルトニアン$H$が$Z$の場合には、期待値の計算ができますが、XやYの場合には求めることができません。そこでハミルトニアンの変形を利用します。

$$
X = HZH
$$

より、

$$
\langle \psi \mid X \mid \psi \rangle \\
= \langle \psi \mid HZH \mid \psi \rangle\\
= \langle \psi' \mid Z \mid \psi' \rangle
$$

のように状態ベクトルを変形することで対応できます。Yの場合は、

$$
Y = RX(-\pi/2) Z RX(\pi/2)
$$

と、なり同様に、


$$
\langle \psi \mid Y \mid \psi \rangle \\
= \langle \psi \mid RX(-\pi/2) Z RX(\pi/2) \mid \psi \rangle\\
= \langle \psi'' \mid Z \mid \psi'' \rangle
$$

とすることができます。これらは、量子回路の測定の直前に上記のHやRXゲートを入れて測定を行うことで期待値を計算することができます。


##2-1-12 線形結合
実際にはハミルトニアン$H$はエルミート行列で与えられ、ユニタリ行列の和の形で与えられ、下記のように分解できます。

$$
\langle \psi \mid aH_1 + bH_2 \mid \psi \rangle \\ = \langle \psi \mid aH_1 \mid \psi \rangle + \langle \psi \mid bH_2 \mid \psi \rangle \\ = a\langle \psi \mid H_1 \mid \psi \rangle + b\langle \psi \mid H_2 \mid \psi \rangle
$$

これを使えば、複雑なハミルトニアンの場合でも、

$$
H = 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1
$$

期待値は、

$$
\langle \psi \mid 1.2 X_0 Z_2 + 2.5 Z_0 X_1 Y_2 - 3.4 Z_2 X_1 \mid \psi \rangle\\
= 1.2\langle \psi \mid X_0 Z_2 \mid \psi \rangle + 2.5 \langle \psi \mid Z_0 X_1 Y_2\mid \psi \rangle - 3.4 \langle \psi \mid Z_2 X_1 \mid \psi \rangle
$$

のように個別に計算をして和を求めることで計算できます。


##2-1-13 例：任意のansatzで実装
ハミルトニアン$H$に対して対応したansatzを使わないと太平洋から一円玉を見つけるような作業になってしまいますが、ここでは例題として１量子ビットのハミルトニアン$H$の固有値を求めてみます。求めたいハミルトニアン$H$は、

```python
from blueqat.pauli import X, Y, Z, I

h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
```

のように作ってみました。読み込まれたのはパウリゲートで行列が対応します。パウリゲートの後ろの数字はそれぞれを適用する量子ビットの通し番号です。上記のハミルトニアン$H$は参考として行列表記は、

```python
[[ 3.57,   -4.56-2.45j]
 [-4.56+2.45j, -1.11  ]]
```

となります。そして、ansatzはこれから探しますが探し方はパラメータを割り振ってそれを探します。1量子ビットの任意の状態ベクトルは極座標で２つの角度があれば表現できます。

```python
Circuit().ry(a)[0].rz(b)[0]
```

としてみます。上記は0番目の量子ビットに、Ryゲートを角度aで、Rzゲートを角度bで適用します。aとbでいい値が見つかれば、上記のansatzはハミルトニアンの期待値に対していい答えを返します。

全体の実装は、



In [3]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 2)
        self.step = 1

    def get_circuit(self, params):
        a, b = params
        return Circuit().ry(a)[0].rz(b)[0] #パラメータでansatzを作る


# ハミルトニアン
h = 1.23 * I - 4.56 * X(0) + 2.45 * Y(0) + 2.34 * Z(0)
runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# これ以降は確認のために
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-4.450814236864202
Result by numpy
-4.450818602983201


となり、ほぼ近い値が固有値として求まっているのがわかります。実際には、上記のように任意でハミルトニアン$H$を書くことは多くないので定式化にルールを定めます。今後は全てblueqatで自動でやってくれますので、理解だけすれば実装は下記のように非常に簡単に実装できます。

In [2]:
from blueqat import vqe
from blueqat.pauli import Z

H = Z(0)*Z(1) #ハミルトニアン
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(H, step)).run()
print(result.most_common(12))

(((1, 0), 0.499883822557386), ((0, 1), 0.499883822557386), ((0, 0), 0.00011617744261394092), ((1, 1), 0.00011617744261394092))


これ以降はansatzとハミルトニアンの関連に触れながら実問題を見ていきます。実問題でこれからみるのは、

１、量子化学計算  
２、組合せ最適化問題

になります。